In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import math
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [ ]:
url_low = 'https://raw.githubusercontent.com/TZstatsADS/Fall2020-Project4-group-8/main/data/lowDim_dataset.csv?token=AOSJD5EUXUPUMLHW4NMK5G27X7LJO'
url_high = 'https://raw.githubusercontent.com/TZstatsADS/Fall2020-Project4-group-8/main/data/highDim_dataset.csv?token=AOSJD5FE2I6YBQ4XODO6LE27X7LOM'
url_pro_low = 'https://raw.githubusercontent.com/TZstatsADS/Fall2020-Project4-group-8/main/output/low_dim_propensity_scores.csv?token=AOSJD5BQYGP4GKLYSWOVOD27X7LSI'
url_pro_high = 'https://raw.githubusercontent.com/TZstatsADS/Fall2020-Project4-group-8/main/output/high_dim_propensity_scores.csv?token=AOSJD5EO6552YHOHZQERQ327X7LUG'
lowDim_dataset = pd.read_csv(url_low)
highDim_dataset = pd.read_csv(url_high)
propensity_low = pd.read_csv(url_pro_low)
propensity_high = pd.read_csv(url_pro_high)

In [ ]:
def ipw_ate(dataset):
    treated = 0
    controlled = 0
    for i in range(dataset.shape[0]):
        if dataset['A'][i] == 1:
            treated += dataset['Y'][i] * dataset['weight'][i]
        else:
            controlled += dataset['Y'][i] * dataset['weight'][i]

    print(treated - controlled)
    ate = (treated - controlled)/dataset.shape[0]
    return ate

In [ ]:
X=lowDim_dataset.iloc[:,2:24].values
A=lowDim_dataset['A'].values
Y=lowDim_dataset['Y'].values
params = {'learning_rate':[0.001,0.01,0.035,0.1], 'max_depth': [1,2,3], 'n_estimators':[50,100,200],
          'min_samples_leaf':[1,2],'min_samples_split':[2,4]}
gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=5).fit(X, A)
print('param: ', gscv.best_params_)
print('score: ', gscv.best_score_)

In [ ]:
gbm = GradientBoostingClassifier(learning_rate=0.035, max_depth=2, min_samples_leaf=1, min_samples_split=4,
                                 n_estimators=50).fit(X, A)
propensity_low_new = [x[1] for x in gbm.predict_proba(X)]

In [ ]:
lowDim_dataset['score'] = propensity_low_new
lowDim_dataset['weight'] = lowDim_dataset['A']/lowDim_dataset['score'] + (1 - lowDim_dataset['A'])/(1 - lowDim_dataset['score'])

In [ ]:
ate_low = ipw_ate(lowDim_dataset)
ate_low

In [ ]:
lowDim_dataset['score'] = propensity_low['propensity_scores']
lowDim_dataset['weight'] = lowDim_dataset['A']/lowDim_dataset['score'] + (1 - lowDim_dataset['A'])/(1 - lowDim_dataset['score'])